<a href="https://colab.research.google.com/github/keshav123333/Machine_learning_100_campusx/blob/main/Ml_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Main archtcture hoga humara

age ,embarked pe
missing value handle -> one hot encode -> scaling -> feauture selection -> model

In [8]:
import pandas as pd

In [18]:
df=pd.read_csv("https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/refs/heads/main/day29-sklearn-pipelines/train.csv")

In [19]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

In [ ]:
xtrain,xest,ytrain,ytest=train_test_split(df.drop(columns=["Survived"]),df["Survived"],test_size=0.2,random_state=42)

In [ ]:
xtrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [ ]:
#impute transformer for cat and num col

trf1=ColumnTransformer([
    ("impute_age",SimpleImputer(),[2]),#yaha pe tune 2 isliye as if name se ppipline mein col toh ftne lagta aage jake
    ("impute_emb",SimpleImputer(strategy="most_frequent"),[6])

    ]
                    , remainder="passthrough")

In [ ]:
trf2=ColumnTransformer([
    ("ohe_sex_embarked",OneHotEncoder(handle_unknown='ignore'),[1,6]),
],remainder="passthrough")

In [ ]:
trf3=ColumnTransformer([
    ("scale",MinMaxScaler(),slice(0,10))# ye cheez as slice sab pe laga dega ye and main cheez  but 10 row aise
    #so sex ebark hatega and uski jagah age ke liye 2 col add and emb ke liye do col add ho jayega so 5 +5 10 honge upar ke steps kebaad
])

In [ ]:
trf4=SelectKBest(score_func=chi2,k=5)# so ye top ke 5 feature ko hi slect kar leg

In [ ]:
trf5=DecisionTreeClassifier()

#Create pipline

In [ ]:
pipe=Pipeline([
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("trf4",trf4),

    ("trf5",trf5)
])

In [ ]:
#second way same cheez karne ka
pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)
#dono mein se koi bhi kar skta hai

In [ ]:
pipe.fit(xtrain,ytrain) # yaha pe model bhi train so ye call

# imp pipe.fit_transform(xtrain,ytrain) # if tu maan le tere pipeline mein sirf one hot min max smple imputer ki hi function so zar

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

In [ ]:
trf2=Pipeline([
    ("scale",MinMaxScaler())# ye cheez as slice sab pe laga dega ye
])

In [ ]:
imputer_age=Pipeline([
    ("impute_age",SimpleImputer())
])

impute_emb=Pipeline([
    ("impute_emb",SimpleImputer(strategy="most_frequent"))
])

ohe_sex_embarked=Pipeline([
    ("ohe_sex_embarked",OneHotEncoder(handle_unknown='ignore'))
])

trf1=ColumnTransformer([
    ("imputer_age",imputer_age,["Age"]),
    ("impute_emb",impute_emb,["Embarked"]),
    ("ohe_sex_emb",ohe_sex_embarked,["Sex","Embarked"]),

])


In [ ]:
pipe=Pipeline([
    ("trf1",trf1),

    ("trf3",trf4),
    ("trf4",trf5)
])

In [ ]:
pipe.fit(xtrain,ytrain)

ValueError: could not convert string to float: 'male'

Ahh samajh gaya 😎 — tu basically **“impute + encode → fir specific columns pe scaler lagana”** chah raha hai. Iska matlab:

1. `trf1` → ColumnTransformer:

   * `Age` → mean imputer
   * `Embarked` → most frequent imputer
   * `Sex + Embarked` → OneHotEncoder

2. `MinMaxScaler` → **only encoded numeric columns** pe lagana (jo output me 0 se 7 index tak ho gaye)

Problem ye hai ki **ColumnTransformer ka output NumPy array hota hai**, aur usme **original column names nahi hote**, isliye ab slice `[0:7]` directly scale ke liye use nahi kar sakte.

---

### 🔹 Solution 1: `ColumnTransformer` + nested pipeline

Hum **numeric columns ke liye scaler pipeline** bana dete hain aur ColumnTransformer me directly use karte hain:

```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Pipelines
imputer_age = SimpleImputer(strategy='mean')
imputer_emb = SimpleImputer(strategy='most_frequent')
ohe_sex_embarked = OneHotEncoder(handle_unknown='ignore')

# Numeric pipeline for Age (impute + scale)
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Categorical pipeline for Embarked (impute) + Sex+Embarked (OHE)
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # for Embarked
    ('ohe', OneHotEncoder(handle_unknown='ignore'))        # Sex + Embarked
])

# Combine in ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, ['Age']),
    ('cat', categorical_pipeline, ['Sex', 'Embarked'])
])
```

✅ **Benefits**:

* `Age` → impute + scale
* `Embarked` → impute → encoded inside pipeline
* `Sex + Embarked` → OHE automatically
* ColumnTransformer output → all numeric, ready for SelectKBest / model

---

### 🔹 Solution 2: Scale only specific output indices (advanced)

Agar tu **slice 0:7** pe lagana chahta hai:

```python
from sklearn.preprocessing import FunctionTransformer
import numpy as np

# Custom scaler for specific columns
def scale_slice(X):
    X = X.copy()
    X[:, 0:7] = MinMaxScaler().fit_transform(X[:, 0:7])
    return X

slice_scaler = FunctionTransformer(scale_slice)
```

* Fir pipeline me use kar sakte ho:

```python
pipe = Pipeline([
    ('preprocessor', trf1),
    ('slice_scale', slice_scaler)
])
```

✅ Ye approach **output array ke specific columns** pe scaler lagane me kaam aata hai.

---

💡 **Takeaway:**

* ColumnTransformer me **numeric pipeline + scaler** use karna sabse clean way hai
* Agar manually slice karna hai → FunctionTransformer ka use karo

---

Agar tu chahe, mai tujhe **full pipeline example** bana ke dikha du jisme:

* `Age` impute + scale
* `Embarked` impute + encode
* `Sex` encode
* SelectKBest
* DecisionTree

Aur **exactly slice 0:7 pe scaler apply ho**.

Chahiye mai bana du?


#yaha se maine main likhna suru kiya hai

In [20]:
df1=pd.read_csv("https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/refs/heads/main/day29-sklearn-pipelines/train.csv")

In [21]:
df1.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

In [22]:
df1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [23]:
df1.isna().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2


In [24]:
xtrain,xest,ytrain,ytest=train_test_split(df1.drop(columns=["Survived"]),df1["Survived"],test_size=0.2,random_state=42)

In [25]:
impute_age=Pipeline([
    ("impute_age",SimpleImputer())
])

impute_ohe=Pipeline([
    ("impute_emb",SimpleImputer(strategy="most_frequent")),
    ("ohe_sex_embarked",OneHotEncoder(handle_unknown="ignore"))
])

trf1=ColumnTransformer([
    ("impute_age",impute_age,["Age"]),
    ("ohe",impute_ohe,["Embarked","Sex"])
])



In [26]:
trf2=MinMaxScaler()

trf3=SelectKBest(score_func=chi2,k=5)

trf4=DecisionTreeClassifier()

In [27]:
pipe=Pipeline([
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("trf4",trf4)
])

In [28]:
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('trf1',
                 ColumnTransformer(transformers=[('impute_age',
                                                  Pipeline(steps=[('impute_age',
                                                                   SimpleImputer())]),
                                                  ['Age']),
                                                 ('ohe',
                                                  Pipeline(steps=[('impute_emb',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe_sex_embarked',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('trf2', MinMaxScaler()),
                ('trf3',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7d2d2f71f6a0>)),
                ('trf4', DecisionTreeClassifier())])

In [29]:
pipe.named_steps

{'trf1': ColumnTransformer(transformers=[('impute_age',
                                  Pipeline(steps=[('impute_age',
                                                   SimpleImputer())]),
                                  ['Age']),
                                 ('ohe',
                                  Pipeline(steps=[('impute_emb',
                                                   SimpleImputer(strategy='most_frequent')),
                                                  ('ohe_sex_embarked',
                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                  ['Embarked', 'Sex'])]),
 'trf2': MinMaxScaler(),
 'trf3': SelectKBest(k=5, score_func=<function chi2 at 0x7d2d2f71f6a0>),
 'trf4': DecisionTreeClassifier()}

In [30]:
pipe.named_steps['trf1'].transformers_[0][1].named_steps['impute_age'].statistics_ #isse hum pehle imputer ki mean nikal sakte

array([29.49884615])

In [31]:
ypred=pipe.predict(xest)

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,ypred)

0.7206703910614525

cross val scorel

In [33]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,xtrain,ytrain,cv=5,scoring="accuracy").mean()

np.float64(0.7613020782034867)

#GridSearch using pipline

In [42]:
params={
    'trf4__max_depth':[1,2,3,4,5,None]   # isme hum trf4 jo descion tree uske parameter ko  twek krne ke liye use kiya hai
}




In [43]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,params,cv=5,scoring="accuracy")
grid.fit(xtrain,ytrain)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(transformers=[('impute_age',
                                                                         Pipeline(steps=[('impute_age',
                                                                                          SimpleImputer())]),
                                                                         ['Age']),
                                                                        ('ohe',
                                                                         Pipeline(steps=[('impute_emb',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe_sex_embarked',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked',
                                                                          'Sex'])])),
                                       ('trf2', MinMaxScaler()),
                                       ('trf3',
                                        SelectKBest(k=5,
                                                    score_func=<function chi2 at 0x7d2d2f71f6a0>)),
                                       ('trf4', DecisionTreeClassifier())]),
             param_grid={'trf4__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

#model dump and use

In [44]:
import pickle

In [45]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [46]:
pipe=pickle.load(open('pipe.pkl','rb'))

In [ ]:
#and ab use pipe.predict everything works fine now